# Metaclasses

[https://realpython.com/python-metaclasses/]

What is metaprogramming / metaclasses?

**Metaprogramming** refers to teh ability of a program to 'know' itself, and be able to manipulate itself. The way Python implements this is by Metaclasses.

Number 1 lesson: you don't need to use metaclasses. But it is helpful to know how metaclasses work

### Old and new style classes
In old-style, `class` and `type` were different things. In new-style, the ideas of a class and a type are unified.

In [1]:
class Foo:
    pass

obj = Foo()
print(obj.__class__)
print(type(obj))
obj.__class__ == type(obj)

<class '__main__.Foo'>
<class '__main__.Foo'>


True

in old style, type(obj) would've been 'instance'. In Python 3 `obj.__class__ == type(obj)` for any object, simple or complex, you can specify.

Remember, everything is an object. Classes are objects, and the 'type' of Classes is `type`. 

In [2]:
print('Foo -->',type(Foo))
x = Foo()
print('x -->',type(x))

# the same holds for builtin classes
print('int -->',type(int))

# and, inception style, for type itself
print('type -->',type(type))

Foo --> <class 'type'>
x --> <class '__main__.Foo'>
int --> <class 'type'>
type --> <class 'type'>


`type` is an object, and the type is `type`

`type` is a metaclass, of which classes are instances. Every class is an instance of the `type` metaclass.

So

* `x` is an instance of class `Foo`
* `Foo` is an instance of metaclass `type`
* `type` is and instance of metaclass `type`

## Dynamically defining a class

the function `type()` when passed one argument, returns the type of an object (generally `obj.__class__`, as we saw above).

You can calso call `type()` with 3 args, `type(<name>, <bases>, <dct>)`, which creates a new instance of the `type` metaclass. **i.e. it dynamically creates a new class**

* `<name>` specifies the name class it becomes `__name__` attr
* `<bases>` specifies a tuple of the base classes from which the class inherits. It becomes `__bases__`
* `<dct>` specifies a namespace dict containing definitions for the class body. It becomes `__dict__`

### Example 1
Create a class `Foo` with 2 objects, `attr` and  `attr_val`
#### using `type()`

In [3]:
Foo = type('Foo',(),{'attr':100, 'attr_val':lambda x:x.attr*5}) # objects are passed via namespace

x=Foo()
print(x.attr)
print(x.attr_val())

100
500


#### Trad way

In [4]:
class Foo:
    attr = 100
    def attr_val(self):
        return self.attr*5

x=Foo()
print(x.attr)
print(x.attr_val())

100
500


### Example with inheritance

In [5]:
Bar = type('Bar',(Foo,),{'attr_bar':200})

y = Bar()
print(y.attr)
print(y.attr_val())
print(y.attr_bar)

100
500
200


### Example with more function passed without lambda


In [6]:
def print_attr(obj):
    print('attr=', obj.attr)
    
Foo = type('Foo',(),{'attr':100,'print_attr':print_attr})

x=Foo()
x.print_attr()


attr= 100


## Custom Metaclasses

What happens when you input `f = Foo()`? You call the `__call__()` method of the `Foo` class, which in turn calls the `__new__()` and `__init__()` methods.

by default these are inherited from `Foo`'s parent class, but the can be overwritten

In [7]:
def new(cls):
    x = object.__new__(cls)
    x.attr = 100
    return x

class Foo:
    pass

Foo.__new__ = new

f = Foo()
f.attr


100

(note typically it is `__init__` that is modified, not `__new__`)

If you tried to follow the pattern so that *every* new Class you created had the `attr = 100`, you might thing you could set `type.__new__ = new`. But that doesn't work: Python blocks you from modifying attributes of type `type`.

If you really want to mess around with how new classes are created, you would create a new meta class, deriving from `type` (which automatically makes `Meta` a metaclass). 

To create new class you would specify that `metaclass=Meta`)

**A reminder that you should not actually do this in your code**

In [8]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        new_class = super().__new__(cls, name, bases, dct)
        new_class.attr = 100
        return new_class
    
class Foo(metaclass = Meta):
    pass

Foo.attr

100

## Factory concept
You can think of a Class as a 'factory' which produces objects.

In the same way, a Metaclass is a factory for producing classes, a 'class factory'

## What's the point?
As mentioned a couple of times, you should **not use custom Metaclasses in your code**. There's usually a better way to achieve the same thing.

The example above can easily be replicated by simple inheritance, or class decorators:

In [9]:
def attr_adder(cls):
    class NewClass(cls):
        attr = 100
    return NewClass

@attr_adder
class X:
    pass

X.attr

100